In [25]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
# from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFECV
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn import feature_selection
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
def get_player_data():
    # initialise dataframes
    df_2021 = pd.read_excel('data/season_stats_2021.xlsx')
    df_2020 = pd.read_excel('data/season_stats_2020.xlsx')
    df_2019 = pd.read_excel('data/season_stats_2019.xlsx')
    df_2018 = pd.read_excel('data/season_stats_2018.xlsx')
    df_2017 = pd.read_excel('data/season_stats_2017.xlsx')
    df_2016 = pd.read_excel('data/season_stats_2016.xlsx')

    # add column for season
    df_2021['season'] = 2021
    df_2020['season'] = 2020
    df_2019['season'] = 2019
    df_2018['season'] = 2018
    df_2017['season'] = 2017
    df_2016['season'] = 2016

    # add missing columns from ingestion
    df_2017['IT'] = np.NaN
    df_2017['KD'] = np.NaN
    df_2016['IT'] = np.NaN
    df_2016['KD'] = np.NaN

    # remove extra column used for ingestion
    # df_2016 = df_2016.drop(columns=['filler_col_1'])

    # combine seasons and cleanse
    df_stats = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020, df_2021])
    df_stats = df_stats.replace('na', np.NaN)
    df_stats = df_stats[df_stats['Rd..'].notna()]

    # col used for joining datasets
    df_stats['join_key'] = df_stats['season'].astype(str) + '_' \
                        + df_stats['Rd..'].astype(str) + '_' \
                        + df_stats[['Team', 'Vs']].max(axis=1) \
                        + df_stats[['Team', 'Vs']].min(axis=1)

    # col used for aggregating match data
    df_stats['group_key'] = df_stats['season'].astype(str) + '_' \
                        + df_stats['Rd..'].astype(str) + '_' \
                        + df_stats['Team']
    
    # aggregrate desired stats by round
    cols = ['Base', 'Score', 'Create', 'Evade', 'Neg', 'TR', 'TS', 'LT', 'GO', 'MG', 'FG', 'MF', 'TA', 'MT', 'TB', 'FD', 'OL', 'IO', 'LB', 'LA', 'FT', 'KB', 'H8', 'HU', 'HG', 'IT', 'KD', 'PC', 'ER', 'SS']
    df_stats = df_stats.groupby(['group_key', 'join_key', 'Team'])[cols].sum().reset_index()

    return df_stats

In [3]:
def get_betting_data():
    cols = ['Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Home Odds', 'Draw Odds', 'Away Odds']

    rename_cols = {
        'Date': 'date',
        'Home Team': 'home_team',
        'Away Team': 'away_team',
        'Home Score': 'home_score',
        'Away Score': 'away_score',
        'Home Odds': 'home_odds',
        'Draw Odds': 'draw_odds',
        'Away Odds': 'away_odds'
    }

    team_mapping = {
        'North QLD Cowboys': 'NQC',
        'St George Dragons': 'STG',
        'Canberra Raiders': 'CBR',
        'South Sydney Rabbitohs': 'STH', 
        'Manly Sea Eagles': 'MNL',
        'Melbourne Storm': 'MEL',
        'Newcastle Knights': 'NEW',
        'Brisbane Broncos': 'BRO',
        'Parramatta Eels': 'PAR',
        'Wests Tigers': 'WST',
        'Sydney Roosters': 'SYD',
        'Canterbury Bulldogs': 'BUL',
        'Gold Coast Titans': 'GCT',
        'Penrith Panthers': 'PTH',
        'New Zealand Warriors': 'NZL',
        'Cronulla Sharks': 'SHA'
    }

    # get data
    df_betting = pd.read_excel('http://www.aussportsbetting.com/historical_data/nrl.xlsx', header=1)[cols].rename(columns=rename_cols)
    df_betting = df_betting[(df_betting['date'] > '2016-01-01') & (df_betting['date'] < '2099-01-01')]
    df_betting = df_betting.replace({'home_team': team_mapping, 'away_team': team_mapping})

    # use date fields to derive round number for each match
    df_betting['season'] = pd.DatetimeIndex(df_betting['date']).year.astype(int)
    df_betting['round_start'] = df_betting['date'].dt.to_period('W-TUE').dt.start_time
    df_betting['round'] = df_betting.groupby('season')['round_start'].rank(method='dense')

    # col used for joining datasets
    df_betting['join_key'] = df_betting['season'].astype(str) + '_' \
                        + df_betting['round'].astype(str) + '_' \
                        + df_betting[['home_team', 'away_team']].max(axis=1) \
                        + df_betting[['home_team', 'away_team']].min(axis=1)

    return df_betting

In [4]:
def prepare_nrl_data():
    df_stats = get_player_data()
    df_betting = get_betting_data()
    df_combined = pd.merge(df_stats, df_betting, on='join_key', how='left')

    df_combined['odds'] = np.where(df_combined['Team'] == df_combined['home_team'], df_combined['home_odds'], df_combined['away_odds'])
    df_combined['margin'] = np.where(df_combined['Team'] == df_combined['home_team'], df_combined['home_score'] - df_combined['away_score'], df_combined['away_score'] - df_combined['home_score'])

    non_feature_cols = ['join_key', 'Team', 'round', 'season', 'date', 'home_team', 'away_team']
    feature_cols = ['odds', 'home_score', 'away_score', 'margin', 'Base', 'Score', 'Create', 'Evade', 'Neg', 'TR', 'TS', 'LT', 'GO', 'MG', 'FG', 'MF', 'TA', 'MT', 'TB', 'FD', 'OL', 'IO', 'LB', 'LA', 'FT', 'KB', 'H8', 'HU', 'HG', 'IT', 'KD', 'PC', 'ER', 'SS']
    df_nrl = df_combined[non_feature_cols + feature_cols]

    # tag each column that will be used in feature creation with 'f_' so we can easily grab these columns
    df_nrl = df_nrl.rename(columns={col: 'f_' + col for col in df_nrl if col in feature_cols})
    return df_nrl

In [5]:
# define a function which returns a DataFrame with the expontential moving average for each numeric stat
def create_exp_weighted_avgs(df, span):
    # create a copy of the df with only the game id and the team - we will add cols to this df
    ema_features = df[['join_key', 'Team']].copy()
    
    # get a list of columns we will iterate over
    feature_names = [col for col in df.columns if col.startswith('f_')] 
    
    for feature_name in feature_names:
        feature_ema = (df.groupby('Team')[feature_name]
                         .transform(lambda row: (row.ewm(span=span)
                                                    .mean()
                                                    .shift(2))))
        ema_features[feature_name] = feature_ema
    
    return ema_features

In [6]:
# define a function which finds the elo for each team in each game and returns a dictionary with the game ID as a key and the
# elos as the key's value, in a list. it also outputs the probabilities and a dictionary of the final elos for each team
def elo_applier(df, k_factor):
    # initialise a dictionary with default elos for each team
    elo_dict = {team: 1500 for team in df['Team'].unique()}
    elos, elo_probs = {}, {}
    
    # get a home and away dataframe so that we can get the teams on the same row
    home_df = df.loc[df.home_team == df.Team, ['Team', 'join_key', 'f_margin']].rename(columns={'Team': 'home_team'})
    away_df = df.loc[df.away_team == df.Team, ['Team', 'join_key']].rename(columns={'Team': 'away_team'})
    
    df = (pd.merge(home_df, away_df, on='join_key')
            .sort_values(by='join_key')
            .drop_duplicates(subset='join_key', keep='first')
            .reset_index(drop=True))

    # Loop over the rows in the DataFrame
    for index, row in df.iterrows():
        # Get the Game ID
        game_id = row['join_key']
        
        # Get the margin
        margin = row['f_margin']
        
        # If the game already has the elos for the home and away team in the elos dictionary, go to the next game
        if game_id in elos.keys():
            continue
        
        # Get the team and opposition
        home_team = row['home_team']
        away_team = row['away_team']
        
        # Get the team and opposition elo score
        home_team_elo = elo_dict[home_team]
        away_team_elo = elo_dict[away_team]
        
        # Calculated the probability of winning for the team and opposition
        prob_win_home = 1 / (1 + 10**((away_team_elo - home_team_elo) / 400))
        prob_win_away = 1 - prob_win_home
        
        # Add the elos and probabilities our elos dictionary and elo_probs dictionary based on the Game ID
        elos[game_id] = [home_team_elo, away_team_elo]
        elo_probs[game_id] = [prob_win_home, prob_win_away]
        
        # Calculate the new elos of each team
        if margin > 0: # Home team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(1 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0 - prob_win_away)
        elif margin < 0: # Away team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(1 - prob_win_away)
        elif margin == 0: # Drawn game' update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0.5 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0.5 - prob_win_away)
        
        # Update elos in elo dictionary
        elo_dict[home_team] = new_home_team_elo
        elo_dict[away_team] = new_away_team_elo

    return elos, elo_probs, elo_dict

In [19]:
def prepare_nrl_features():
    df_nrl = prepare_nrl_data()
    
    # append next week's games
    df_nrl = get_next_week(df_nrl)

    features = df_nrl[['join_key', 'Team', 'date', 'home_team', 'away_team']].copy()
    features_rolling_averages = create_exp_weighted_avgs(df_nrl, span=10)
    features = pd.merge(features, features_rolling_averages, on=['join_key', 'Team'])

    # use the elo applier function to get the elos and elo probabilities for each game - we will map these later
    elos, probs, elo_dict = elo_applier(df_nrl, 30)

    one_line_cols = ['join_key', 'Team'] + [col for col in features if col.startswith('f_')]

    # get all features onto individual rows for each match
    features_one_line = (features.loc[features.Team == features.home_team, one_line_cols]
                                    .rename(columns={'Team': 'home_team'})
                                    .pipe(pd.merge, (features.loc[features.Team == features.away_team, one_line_cols]
                                    .rename(columns={'Team': 'away_team'})
                                    .rename(columns={col: col+'_away' for col in features.columns if col.startswith('f_')})), on='join_key'))

    # add our created features - elo, efficiency etc.
    features_one_line = (features_one_line.assign(f_elo_home=lambda df: df.join_key.map(elos).apply(lambda x: x[0]),
    f_elo_away=lambda df: df.join_key.map(elos).apply(lambda x: x[1]))
                                        .pipe(pd.merge, df_nrl.loc[df_nrl.Team == df_nrl.home_team, ['join_key', 'date', 'round']], on=['join_key'])
                                        .dropna()
                                        .reset_index(drop=True)
                                        .assign(season=lambda df: df.date.apply(lambda row: row.year)))

    # order the columns so that the game info is on the left
    ordered_cols = [col for col in features_one_line if col[:2] != 'f_'] + [col for col in features_one_line if col.startswith('f_')]
    df_feature = features_one_line[ordered_cols]

    # merge feautre df with results
    # match_results = df_betting[['join_key', 'home_score', 'away_score']]
    # match_results['result'] = match_results.apply(lambda x: 1 if x['home_score'] > x['away_score'] else 0, axis=1)
    # df_combined = pd.merge(df_feature, match_results[['join_key', 'result']], on='join_key')

    # create differential df - this df is the home features minus the away features
    diff_cols = [col for col in df_feature.columns if col + '_away' in df_feature.columns and col != 'f_odds' and col.startswith('f_')]
    non_diff_cols = [col for col in df_feature.columns if col not in diff_cols and col[:-5] not in diff_cols]

    diff_df = df_feature[non_diff_cols].copy()

    for col in diff_cols:
        diff_df[col+'_diff'] = df_feature[col] - df_feature[col+'_away']

    odds = get_betting_data()

    odds['f_current_odds_prob'] = 1 / odds['home_odds']
    odds['f_current_odds_prob_away'] = 1 / odds['away_odds']

    diff_df = pd.merge(diff_df, odds[['join_key', 'f_current_odds_prob', 'f_current_odds_prob_away']], on='join_key')

    return diff_df

In [29]:
def get_next_week(df):
    df_fixture = pd.read_csv('data/nrl_2021_fixture.csv')

    team_mapping = {
        'Cowboys': 'NQC',
        'Dragons': 'STG',
        'Raiders': 'CBR',
        'Rabbitohs': 'STH', 
        'Sea Eagles': 'MNL',
        'Storm': 'MEL',
        'Knights': 'NEW',
        'Broncos': 'BRO',
        'Eels': 'PAR',
        'Wests Tigers': 'WST',
        'Roosters': 'SYD',
        'Bulldogs': 'BUL',
        'Titans': 'GCT',
        'Panthers': 'PTH',
        'Warriors': 'NZL',
        'Sharks': 'SHA'
    }

    rename_cols = {
        'Date': 'date',
        'Round Number': 'round',
        'Home Team': 'home_team',
        'Away Team': 'away_team'
    }

    df_fixture = df_fixture.replace({'Home Team': team_mapping, 'Away Team': team_mapping}).rename(columns=rename_cols)
    df_fixture['season'] = 2021
    df_fixture['date'] = datetime.fromisoformat('2099-01-01')

    # col used for joining datasets
    df_fixture['join_key'] = '2021' + '_' \
                    + df_fixture['round'].astype(str) + '.0' '_' \
                    + df_fixture[['home_team', 'away_team']].max(axis=1) \
                    + df_fixture[['home_team', 'away_team']].min(axis=1)

    df_fixture = df_fixture[['join_key', 'home_team', 'away_team', 'round', 'season', 'date']]
    df_fixture = df_fixture[df_fixture['round'] == df[df['season']==2021]['round'].max() + 1]

    h_fixture = df_fixture.copy()
    a_fixture = df_fixture.copy()

    h_fixture['Team'] = h_fixture['home_team']
    a_fixture['Team'] = a_fixture['away_team']


    # append next round fixtures to nrl df
    # df = df.append(df_fixture[df_fixture['round'] == df[df['season']==2021]['round'].max() + 1]).reset_index(drop=True)
    df = pd.concat([df, h_fixture, a_fixture])

    return df

In [9]:
# merge feautre df with results
match_results = df_betting[['join_key', 'home_score', 'away_score']]
match_results['result'] = match_results.apply(lambda x: 1 if x['home_score'] > x['away_score'] else 0, axis=1)
feature_df = pd.merge(df_feature, match_results[['join_key', 'result']], on='join_key')
feature_df.tail()

NameError: name 'df_betting' is not defined

In [30]:

nrl_df = prepare_nrl_features()
nrl_df.tail(10)






,join_key,home_team,away_team,date,round,season,f_odds,f_odds_away,f_elo_home,f_elo_away,f_Base_diff,f_Create_diff,f_ER_diff,f_Evade_diff,f_FD_diff,f_FG_diff,f_FT_diff,f_GO_diff,f_H8_diff,f_HG_diff,f_HU_diff,f_IO_diff,f_IT_diff,f_KB_diff,f_KD_diff,f_LA_diff,f_LB_diff,f_LT_diff,f_MF_diff,f_MG_diff,f_MT_diff,f_Neg_diff,f_OL_diff,f_PC_diff,f_SS_diff,f_Score_diff,f_TA_diff,f_TB_diff,f_TR_diff,f_TS_diff,f_away_score_diff,f_home_score_diff,f_margin_diff,f_current_odds_prob,f_current_odds_prob_away
974,2021_8.0_PTHMNL,PTH,MNL,2021-05-01,8.0,2021,1.388554,3.360933,1704.272203,1444.415293,30.738303,39.387031,-0.135184,53.966769,6.488862,1.422340,1.643675,6.504022,36.361383,-0.577794,-6.795466,-3.425651,0.258781,9.737040e-01,-0.561584,13.045141,27.172505,1.785368,-0.089950,-0.945260,-1.845643,1.920231,5.971739,1.054678,1.000737,38.462896,3.018029,23.989396,32.149536,16.011865,4.502742,-7.734713,24.663792,0.862069,0.188679
975,2021_8.0_WSTSTG,STG,WST,2021-05-02,8.0,2021,2.596876,3.646237,1456.920579,1380.542073,31.165171,-1.330372,-1.386446,-39.336174,-2.369742,0.294184,-0.029179,2.790358,6.174220,-0.325747,-3.983090,0.047776,-1.354745,1.599835e-01,-0.084482,-1.440341,-10.741569,-2.293574,0.171698,0.547942,0.972252,4.182560,-11.918852,4.529264,1.039742,5.901437,28.001789,-15.368785,2.423003,4.726962,-5.109301,-1.179997,11.456119,0.763359,0.288184
976,2021_9.0_STHMEL,STH,MEL,2099-01-01,9.0,2099,1.473001,1.436426,1643.860725,1732.209437,-24.758231,-6.456327,0.235465,-28.001074,-3.220079,3.131596,2.455940,0.136179,-0.649416,0.829038,-12.549510,-2.603168,-0.466865,4.658914e-08,-0.757505,-5.770374,-7.451765,2.875533,-0.385291,0.324590,-2.217986,-0.883048,-11.602556,-0.263573,-0.854940,-12.355364,-9.341318,-5.876719,-16.391476,-2.039842,3.598005,-3.537528,-3.111822,0.269542,0.775194
977,2021_9.0_SHAPTH,PTH,SHA,2099-01-01,9.0,2099,1.336090,2.185340,1709.763532,1459.880371,118.056405,-3.653457,0.797086,27.403612,1.583886,1.815802,1.067152,3.337104,36.145918,-0.074589,5.403676,-2.342826,0.001970,7.074586e-01,-0.761712,-4.403088,12.467527,-0.973551,-0.025035,-0.567777,0.525807,3.981306,10.873165,1.215408,1.968813,13.562063,75.981005,6.403777,9.076557,-0.873602,-2.256821,-5.815498,18.194926,0.934579,0.119617
978,2021_9.0_SYDPAR,PAR,SYD,2099-01-01,9.0,2099,1.620267,1.662008,1656.296300,1678.499991,39.750787,20.240516,-0.561902,7.000979,0.283719,0.710608,1.035143,5.778786,38.472225,4.967543,-6.680829,1.177372,-0.259282,-4.452585e-01,-0.559803,3.417313,-6.608889,3.698721,-0.028265,0.116503,0.665638,5.520577,3.890663,3.092475,2.990003,18.164049,7.293752,8.801115,6.618873,12.810682,2.989010,-3.347306,3.026851,0.675676,0.364964
979,2021_9.0_NEWCBR,CBR,NEW,2099-01-01,9.0,2099,1.740862,2.966969,1540.434840,1453.156846,-89.154154,-26.926257,2.575614,-19.651536,-4.860611,-0.584695,-1.356138,0.095353,-19.835064,1.092289,-5.030633,-1.478524,-0.609339,-3.935738e-09,-0.067145,-10.591985,-10.184189,-3.204715,-0.032940,0.792861,-0.370353,0.689088,-8.848507,-0.983567,-0.902960,-11.095968,-63.918103,1.469022,-12.458836,-6.845663,5.503609,-4.869389,2.448563,0.675676,0.366300
980,2021_9.0_WSTGCT,WST,GCT,2099-01-01,9.0,2099,3.430558,2.187670,1398.787486,1371.287999,11.455773,-23.630558,4.539962,-28.818959,-2.447390,0.095341,-0.564637,-2.565887,-17.081775,-3.608002,18.352760,-0.399953,1.017238,-8.672824e-04,0.076653,-12.066294,-21.875481,-2.434359,-0.136023,-0.112488,-4.336223,2.041576,2.893936,-1.953563,-0.544824,-17.883926,14.517070,-10.454699,-11.556867,-6.193664,6.012026,-6.239643,-7.738659,0.400000,0.641026
981,2021_9.0_NQCBRO,NQC,BRO,2099-01-01,9.0,2099,3.260625,6.125075,1370.217181,1358.324230,-17.225831,14.245294,1.464869,-31.149400,2.827551,0.028817,-0.936731,0.731127,1.530473,-5.009082,-19.055480,-1.851945,1.201713,9.736690e-01,0.696115,3.745713,0.975525,-1.926586,0.362770,0.715703,6.090428,1.600628,-18.826803,0.053557,0.082202,20.294550,-5.791252,-12.647890,23.465214,8.865563,10.207636,-0.569600,8.848418,0.613497,0.425532
982,2021_9.0_NZLMNL,MNL,NZL,2099-01-0

In [118]:
feature_df['id'] = feature_df.index + 1

# Create our test and train sets from our afl DataFrame; drop the columns which leak the result, duplicates, and the advanced
# stats which don't have data until 2015
feature_columns = [col for col in feature_df if col.startswith('f_')]

# Create our test set
test_x = feature_df.loc[feature_df.season == 2021, ['id'] + feature_columns]
test_y = feature_df.loc[feature_df.season == 2021, 'result']

# Create our train set
X = feature_df.loc[feature_df.season != 2021, ['id'] + feature_columns]
y = feature_df.loc[feature_df.season != 2021, 'result']

# Scale features
scaler = StandardScaler()
X[feature_columns] = scaler.fit_transform(X[feature_columns])
test_x[feature_columns] = scaler.transform(test_x[feature_columns])

In [119]:
# Create a list of standard classifiers
classifiers = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),
    
    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),
    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
#     XGBClassifier()    
]

# Define a functiom which finds the best algorithms for our modelling task
def find_best_algorithms(classifier_list, X, y):
    # This function is adapted from https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
    # Cross validate model with Kfold stratified cross validation
    kfold = StratifiedKFold(n_splits=5)
    
    # Grab the cross validation scores for each algorithm
    cv_results = [cross_val_score(classifier, X, y, scoring = "neg_log_loss", cv = kfold) for classifier in classifier_list]
    cv_means = [cv_result.mean() * -1 for cv_result in cv_results]
    cv_std = [cv_result.std() for cv_result in cv_results]
    algorithm_names = [alg.__class__.__name__ for alg in classifiers]
    
    # Create a DataFrame of all the CV results
    cv_results = pd.DataFrame({
        "Mean Log Loss": cv_means,
        "Log Loss Std": cv_std,
        "Algorithm": algorithm_names
    })
    
    
    return cv_results.sort_values(by='Mean Log Loss').reset_index(drop=True)

In [120]:
best_algos = find_best_algorithms(classifiers, X, y)
best_algos

,Mean Log Loss,Log Loss Std,Algorithm
0,0.632071,3.371373e-02,LogisticRegressionCV
1,0.654417,2.465576e-02,ExtraTreesClassifier
2,0.659458,3.263966e-02,RandomForestClassifier
3,0.672034,4.109559e-02,LinearDiscriminantAnalysis
4,0.681191,1.109935e-02,NuSVC
5,0.687930,2.510764e-02,SVC
6,0.693147,4.347706e-11,GaussianProcessClassifier
7,0.700895,1.768763e-02,AdaBoostClassifier
8,0.703063,3.520824e-02,GradientBoostingClassifier
9,1.070915,2.770051e-01,BaggingClassifier


In [121]:
# Try a logistic regression model and see how it performs in terms of accuracy
kfold = StratifiedKFold(n_splits=5)
cv_scores = cross_val_score(linear_model.LogisticRegressionCV(), X, y, scoring='accuracy', cv=kfold)
cv_scores.mean()

0.6336956521739131

In [122]:
df_betting['home_win'] = df_betting.apply(lambda x: 1 if x['home_score'] > x['away_score'] else 0, axis=1)
df_betting[['home_win', 'season']].groupby(['season']).mean()

,home_win
year,
2016,0.582090
2017,0.537313
2018,0.577114
2019,0.557214
2020,0.532544
2021,0.531250


In [1]:
def find_odds_prediction(row):
    if row['home_odds'] < row['away_odds'] and row['home_score'] > row['away_score']:
        return 1
    elif row['away_odds'] < row['home_odds'] and row['away_score'] > row['home_score']:
        return 1
    else:
        return 0

df_betting['odds_prediction'] = df_betting.apply(find_odds_prediction, axis=1)
df_betting[['odds_prediction', 'season']].groupby(['season']).mean()

NameError: name 'df_betting' is not defined

In [124]:
# Define a function which optimises the hyperparameters of our chosen algorithms
def optimise_hyperparameters(train_x, train_y, algorithms, parameters):
    kfold = StratifiedKFold(n_splits=5)
    best_estimators = []
    
    for alg, params in zip(algorithms, parameters):
        gs = GridSearchCV(alg, param_grid=params, cv=kfold, scoring='neg_log_loss', verbose=1)
        gs.fit(train_x, train_y)
        best_estimators.append(gs.best_estimator_)
    return best_estimators

# Define our parameters to run a grid search over
lr_grid = {
    "C": [0.0001, 0.001, 0.01, 0.05, 0.2, 0.5],
    "solver": ["newton-cg", "lbfgs", "liblinear"]
}

# Add our algorithms and parameters to lists to be used in our function
alg_list = [LogisticRegression()]
param_list = [lr_grid]

In [125]:
# Find the best estimators, then add our other estimators which don't need optimisation
best_estimators = optimise_hyperparameters(X, y, alg_list, param_list)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [126]:
lr_best_params = best_estimators[0].get_params()
lr_best_params

{'C': 0.01,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'newton-cg',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [127]:
kfold = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(linear_model.LogisticRegression(**lr_best_params), X, y, scoring='neg_log_loss', cv=kfold)
cv_scores.mean()

-0.6319521151181078

In [128]:
lr = LogisticRegression(**lr_best_params)
lr.fit(X, y)
final_predictions = lr.predict(test_x)

accuracy = (final_predictions == test_y).mean()
accuracy

0.703125

In [179]:
# find incorrectly predicted games
game_ids = test_x[(final_predictions != test_y)].id
game_keys = feature_df.loc[feature_df.id.isin(game_ids), ['join_key']]['join_key'].tolist()
df_betting.loc[df_betting.join_key.isin(game_keys)]

,date,home_team,away_team,home_score,away_score,home_odds,draw_odds,away_odds,year,round_start,round,join_key,home_win,odds_prediction
0,2021-05-02,STG,WST,8,16,1.31,23.35,3.47,2021,2021-04-28,8.0,2021_8.0_WSTSTG,0,0
5,2021-04-30,BRO,GCT,36,28,3.57,23.12,1.30,2021,2021-04-28,8.0,2021_8.0_GCTBRO,1,0
10,2021-04-25,WST,MNL,6,40,2.46,20.67,1.56,2021,2021-04-21,7.0,2021_7.0_WSTMNL,0,1
12,2021-04-24,SHA,BUL,12,18,1.12,30.17,6.27,2021,2021-04-21,7.0,2021_7.0_SHABUL,0,0
14,2021-04-23,GCT,STH,30,40,2.81,21.96,1.45,2021,2021-04-21,7.0,2021_7.0_STHGCT,0,1
17,2021-04-18,STG,NZL,14,20,1.42,22.13,2.91,2021,2021-04-14,6.0,2021_6.0_STGNZL,0,0
20,2021-04-17,MNL,GCT,36,0,3.00,21.88,1.40,2021,2021-04-14,6.0,2021_6.0_MNLGCT,1,0
24,2021-04-11,PAR,STG,12,26,1.26,23.42,3.95,2021,2021-04-07,5.0,2021_5.0_STGPAR,0,0
25,2021-04-11,WST,NQC,30,34,1.18,27.33,4.79,2021,2021-04-07,5.0,2021_5.0_WSTNQC,0,0
30,2021-04-09,NZL,MNL,12,13,1.40,22.13,3.01,2021,2021-04-07,5.0,2021_5.0_NZLMNL,0,0


In [158]:
predictions_probs = lr.predict_proba(test_x)
metrics.log_loss(test_y, predictions_probs)


0.5193818172447264

In [206]:
# append next round fixtures to feature df
df_feature[df_feature['season']==2021]['round'].max()
df_prediction = df_feature.append(df_fixture[df_fixture['round'] == df_feature[df_feature['season']==2021]['round'].max() + 1]).reset_index(drop=True)
df_prediction.tail(10)

,away_team,date,f_Base,f_Base_away,f_Create,f_Create_away,f_ER,f_ER_away,f_Evade,f_Evade_away,f_FD,f_FD_away,f_FG,f_FG_away,f_FT,f_FT_away,f_GO,f_GO_away,f_H8,f_H8_away,f_HG,f_HG_away,f_HU,f_HU_away,f_IO,f_IO_away,f_IT,f_IT_away,f_KB,f_KB_away,f_KD,f_KD_away,f_LA,f_LA_away,f_LB,f_LB_away,f_LT,f_LT_away,f_MF,f_MF_away,f_MG,f_MG_away,f_MT,f_MT_away,f_Neg,f_Neg_away,f_OL,f_OL_away,f_PC,f_PC_away,f_SS,f_SS_away,f_Score,f_Score_away,f_TA,f_TA_away,f_TB,f_TB_away,f_TR,f_TR_away,f_TS,f_TS_away,f_away_score,f_away_score_away,f_elo_away,f_elo_home,f_home_score,f_home_score_away,f_margin,f_margin_away,f_odds,f_odds_away,home_team,join_key,round,season
982,MNL,2021-05-01,582.500948,543.533245,103.123045,83.442747,-20.505673,-23.304159,168.290074,135.044535,13.092681,7.783611,1.818832,0.655100,2.492240,2.965596,16.540258,12.673331,230.520953,193.498003,1.299320,2.862970,59.572358,64.950467,2.002492,4.805297,0.927732,0.716002,0.796667,5.629999e-10,-2.203150,-1.198217,35.465572,24.792275,63.512910,46.735406,7.214908,7.208698,-0.217294,-0.143699,-2.161197,-1.751440,-27.458351,-26.311915,-26.286100,-31.493562,34.723887,29.110646,-5.226458,-6.816649,-0.553969,-1.372754,95.144717,75.129620,319.865987,311.396691,67.123052,53.677183,77.864797,60.833358,46.264127,41.890783,18.278908,20.776664,1444.415293,1704.272203,17.290912,20.346586,17.709958,0.439583,1.336090,3.033491,PTH,2021_8.0_PTHMNL,8.0,2021
983,WST,2021-05-02,576.292804,550.248574,70.537807,66.535383,-21.896812,-20.398810,115.471010,142.746061,11.092614,8.667858,0.336065,0.095369,0.023324,0.047197,12.080277,9.797257,197.488083,192.800085,1.172215,0.893281,72.260934,74.974371,3.634261,3.958808,1.196308,1.395645,0.130896,2.053126e-08,-1.659672,-1.590551,20.718606,21.897067,33.557416,40.527791,5.523879,6.673167,-0.011909,-0.152389,-1.613722,-1.698402,-33.989726,-34.057932,-30.812331,-32.779880,30.744335,38.314305,-6.481918,-10.551317,-2.433601,-1.829753,68.474228,60.373053,340.533513,316.532050,46.338689,58.549513,56.511303,51.437937,34.708160,30.840646,17.948403,27.583286,1380.542073,1456.920579,22.259684,18.134227,-0.484651,-11.676021,2.690171,3.430558,STG,2021_8.0_WSTSTG,8.0,2021
984,MEL,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STH,2021_9.0_STHMEL,9.0,2021
985,SHA,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTH,2021_9.0_SHAPTH,9.0,2021
986,SYD,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAR,2021_9.0_SYDPAR,9.0,2021
987,NEW,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBR,2021_9.0_NEWCBR,9.0,2021
988,GCT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WST,2021_9.0_WSTGCT,9.0,2021
989,BRO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na